In [3]:
!pip install --upgrade "watson-developer-cloud>=2.2.6"

Requirement already up-to-date: watson-developer-cloud>=2.2.6 in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (2.2.6)


In [4]:
%run "apikey.py"

from watson_developer_cloud import ToneAnalyzerV3, WatsonApiException

# version can be 2016-05-19 or 2017-09-21
# 2016 version returns between 0 and 1, 2017 returns between 0.5 and 1
tone_analyzer = ToneAnalyzerV3(
    version='2016-05-19',
    username=api['ToneAnalyzer']['username'],
    password=api['ToneAnalyzer']['password'],
    url=api['ToneAnalyzer']['URL']
)

In [5]:
text = 'Team, I know that times are tough! Product '\
    'sales have been disappointing for the past three '\
    'quarters. We have a competitive product, but we '\
    'need to do a better job of selling it!'

In [6]:
# analyze 'text': only 2500 API calls / month!
try:
    result = tone_analyzer.tone(
        {'text': text},
        'application/json'
    ).get_result()
except WatsonApiException as ex:
    print("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [7]:
def to_tone_array(tone_arr_json):
    """
    turn [dict({'score': {float}, 'tone_id': {str}, 'tone_name', {str}})] * {1 <= int <= 6}
    into [{float}] * 7 where each index is the score of the tone specified in id_to_key
    """
    tone_dict = {}
    for tone in tone_arr_json:
        tone_dict[tone['tone_name']] = tone['score']
    
    return list(tone_dict.values())

In [8]:
def distance(arr1, arr2):
    """
    find distance between two n-dimensional arrays
    """
    return sum([(arr2[i] - arr1[i]) ** 2 for i in range(len(arr1))]) ** (0.5)

In [12]:
doc_tone = result['document_tone']
sentence_tones = result['sentences_tone']
print('document-level tones:')
doc_tones = to_tone_array(doc_tone['tone_categories'][0]['tones'])
print(doc_tones)

sentences = {}
for sentence in sentence_tones:
    sentences[sentence['text']] = to_tone_array(sentence['tone_categories'][0]['tones'])

# prints distance from each sentence to the document-level analysis: to figure out which sentence is 
# the most different in terms of tone from the other sentences
for text in sentences.keys():
    print(text)
    print(sentences[text])
    print(distance(doc_tones, sentences[text]))
    
    
# maybe try a chi-sq test to see which emotion is statistically different?

document-level tones:
[0.081788, 0.008589, 0.076676, 0.039805, 0.6165]
Team, I know that times are tough!
[0.069592, 0.007056, 0.047162, 0.042664, 0.317869]
0.30035115572109927
Product sales have been disappointing for the past three quarters.
[0.10126, 0.025739, 0.119045, 0.018734, 0.771241]
0.1638815961815114
We have a competitive product, but we need to do a better job of selling it!
[0.064169, 0.0379, 0.085963, 0.226295, 0.1997]
0.45799189114109873
